## Outline
1. PyTorch
2. What are tensors
3. Initialising, slicing, reshaping tensors
4. Numpy and PyTorch interfacing
5. GPU support for PyTorch + Enabling GPUs on Google Colab
6. Speed comparisons, NumPy - PyTorch - PyTorch on GPU
7. Autograd concepts and application
8. Writing basic learning loop using autograd
9. Exercises

In [0]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Initialise tensors

Initialise using in-built functions

In [0]:
x = torch.ones(3,2)
print(x)
x = torch.zeros(3,2)
print(x)
x = torch.rand(3,2)
print(x)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0.9766, 0.0918],
        [0.1453, 0.3810],
        [0.4336, 0.8751]])


In [0]:
x = torch.empty(3,2)
# Creates space but doesn't initialise values in it. So just has the values which were there initially in memory (could be NaN too)
print(x)
y = torch.zeros_like(x)
print(y)

tensor([[2.1517e-36, 0.0000e+00],
        [3.3631e-44, 0.0000e+00],
        [       nan, 0.0000e+00]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [0]:
x = torch.linspace(0,1, steps = 5)
# Include 0 and 1
print(x)

tensor([0.0000, 0.2500, 0.5000, 0.7500, 1.0000])


Initialising manually

In [0]:
x = torch.tensor([[1,2],
                  [3,4],
                  [5,6]])
print(x)

tensor([[1, 2],
        [3, 4],
        [5, 6]])


# Slicing tensors

In [0]:
print(x.size()) 
print(x[:,1])
print(x[0,:])

torch.Size([3, 2])
tensor([2, 4, 6])
tensor([1, 2])


In [0]:
y = x[1,1]
print(y) # 
print(y.item()) 

tensor(4)
4


# Reshaping tensors

In [0]:
print(x)
y = x.view(2,3)
print(y)

tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2, 3],
        [4, 5, 6]])


In [0]:
y = x.view(6, -1)
print(y)

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])


# Simple Tensor Operations

In [0]:
# Pointwise Operations
x = torch.ones([3,2])
y = torch.ones([3,2])
z = x + y
print(z)
z = x-y
print(z)
z = x*y
print(z)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [0]:
z = y.add(x)
print(z)
print(y)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [0]:
z = y.add_(x) # Modifies y too (_ => modify-in-place). Adds x to y. z modified as well
print(z)
print(y)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])


# Numpy <> PyTorch

Converting PyTorch to NumPy array

In [0]:
x_np = x.numpy()
print(type(x), type(x_np))
print(x_np)

<class 'torch.Tensor'> <class 'numpy.ndarray'>
[[1. 1.]
 [1. 1.]
 [1. 1.]]


Converting NumPy array to PyTorch tensor

In [0]:
a = np.random.randn(5)
print(a)
a_pt = torch.from_numpy(a)
print(type(a), type(a_pt))
print(a_pt)

[ 0.29573148 -0.15142621  1.43368774  0.96222957 -0.51295352]
<class 'numpy.ndarray'> <class 'torch.Tensor'>
tensor([ 0.2957, -0.1514,  1.4337,  0.9622, -0.5130], dtype=torch.float64)


In [0]:
np.add(a, 1, out=a)
print(a)
print(a_pt)

[1.29573148 0.84857379 2.43368774 1.96222957 0.48704648]
tensor([1.2957, 0.8486, 2.4337, 1.9622, 0.4870], dtype=torch.float64)


We just did bridging, not copying. Therefore change in one affects the other.

In [0]:
%%time
for i in range(100):
  a = np.random.randn(100, 100)
  b = np.random.randn(100, 100)
  c = a + b

CPU times: user 102 ms, sys: 105 µs, total: 103 ms
Wall time: 106 ms


In [0]:
%%time
for i in range(100):
  a = torch.randn([100,100])
  b = torch.randn([100,100])
  c = a + b

CPU times: user 21 ms, sys: 872 µs, total: 21.8 ms
Wall time: 26.5 ms


Still CPU, not GPU

In [0]:
%%time
for i in range(100):
  a = np.random.randn(100, 100)
  b = np.random.randn(100, 100)
  c = np.matmul(a, b)

CPU times: user 163 ms, sys: 117 ms, total: 279 ms
Wall time: 155 ms


In [0]:
%%time
for i in range(100):
  a = torch.randn([100,100])
  b = torch.randn([100,100])
  c = torch.matmul(a,b)

CPU times: user 24.1 ms, sys: 1.09 ms, total: 25.1 ms
Wall time: 84.1 ms


In [0]:
%%time
for i in range(10):
  a = np.random.randn(1000, 1000)
  b = np.random.randn(1000, 1000)
  c = a+b

CPU times: user 965 ms, sys: 10.4 ms, total: 976 ms
Wall time: 978 ms


In [0]:
%%time
for i in range(10):
  a = torch.randn([1000,1000])
  b = torch.randn([1000,1000])
  c = a + b

CPU times: user 169 ms, sys: 887 µs, total: 169 ms
Wall time: 173 ms


Huge improvement from Python list to Numpy.  
Then additional huge improvement from NumPy to PyTorch.  
And now amazingly more using Cuda

# CUDA support

In [14]:
print(torch.cuda.device_count())

1


In [0]:
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [0]:
cuda0 = torch.device('cuda:0')

In [0]:
a = torch.ones(3, 2, device = cuda0)
b = torch.ones(3, 2, device = cuda0)
c = a + b
print(c)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]], device='cuda:0')


In [0]:
print(a)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], device='cuda:0')


In [0]:
%%time
for i in range(10):
  a_cpu = torch.randn([10000,10000])
  b_cpu = torch.randn([10000,10000])
  b_cpu.add(a_cpu)

CPU times: user 14.2 s, sys: 484 ms, total: 14.6 s
Wall time: 14.7 s


In [0]:
%%time
for i in range(10):
  a_gpu = torch.randn([10000,10000],device = cuda0)
  b_gpu = torch.randn([10000,10000],device = cuda0)
  b_gpu.add(a_gpu)

CPU times: user 555 µs, sys: 2.06 ms, total: 2.61 ms
Wall time: 10.3 ms


1000 times improvement!!!

In [0]:
%%time
for i in range(100):
  a_cpu = torch.randn([10000,10000])
  b_cpu = torch.randn([10000,10000])
  torch.matmul(a_cpu,b_cpu)

CPU times: user 22min 33s, sys: 1.05 s, total: 22min 34s
Wall time: 22min 34s


In [0]:
%%time
for i in range(100):
  a_gpu = torch.randn([10000,10000], device = cuda0)
  b_gpu = torch.randn([10000,10000], device = cuda0)
  torch.matmul(a_gpu,b_gpu)

CPU times: user 7 ms, sys: 2 ms, total: 9 ms
Wall time: 13.9 ms


22 minutes versus 13 ms !!!

# Autograd

In [0]:
x = torch.ones([3,2], requires_grad = True) # x is something which could be differentiated against
print(x)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)


In [0]:
y = x + 5
print(y)

tensor([[6., 6.],
        [6., 6.],
        [6., 6.]], grad_fn=<AddBackward0>)


In [0]:
z = y*y + 1
print(z)

tensor([[37., 37.],
        [37., 37.],
        [37., 37.]], grad_fn=<AddBackward0>)


In [0]:
# Similar to forward pass where y is func of x, z is func of y, t is func of z
t = torch.sum(z)
print(t)

tensor(222., grad_fn=<SumBackward0>)


In [0]:
# Now do backward pass since t is func of x
t.backward()

In [0]:
# Prints the Derivative of t w.r.t. x
print(x.grad)

tensor([[12., 12.],
        [12., 12.],
        [12., 12.]])


The derivative is 2yi*1  
Here xi=1 (first matrix) and yi=6 (second matrix)  
Hence x.gradi = 12

In [0]:
x = torch.ones([3,2], requires_grad = True)
y = x+5
r = 1/(1+torch.exp(-y))
print(r)
s = torch.sum(r) # Adds all values of r => tensor with single value in it
print(s)
s.backward()
print(x.grad)

tensor([[0.9975, 0.9975],
        [0.9975, 0.9975],
        [0.9975, 0.9975]], grad_fn=<MulBackward0>)
tensor(5.9852, grad_fn=<SumBackward0>)
tensor([[0.0025, 0.0025],
        [0.0025, 0.0025],
        [0.0025, 0.0025]])


In [0]:
x = torch.ones([3,2], requires_grad = True)
y = x+5
r = 1/(1+torch.exp(-y))
print(r)
a = torch.ones([3,2])#Same size as r
r.backward(a)
print(x.grad)

tensor([[0.9975, 0.9975],
        [0.9975, 0.9975],
        [0.9975, 0.9975]], grad_fn=<MulBackward0>)
tensor([[0.0025, 0.0025],
        [0.0025, 0.0025],
        [0.0025, 0.0025]])


Same result

# Autograd example that looks like what we have been doing

In [0]:
x = torch.randn([20, 1], requires_grad = True) # Training dataset
y = 3*x - 2 # Ground truth model (linear fit)

In [0]:
w = torch.tensor([1.], requires_grad = True) # weight and bias are of size 1
b = torch.tensor([1.], requires_grad = True)

# Predicted model though we know w = 3, b = -2
y_hat = w*x + b

# Forward pass
loss = torch.sum((y_hat - y)**2)

In [5]:
# Not 0 => wrong values of w and b
print(loss)

tensor(299.8250, grad_fn=<SumBackward0>)


In [0]:
loss.backward()

In [7]:
# this time framework computes the grad itself. We don't have to compute the differentiations ourselves
print(w.grad, b.grad)

tensor([-102.9003]) tensor([131.2831])


Loss w.r.t. w is -ve implies increase w (to get the ideal +3)  
Loss w.r.t. b is +ve, so decrease b (to get the ideal -2)

# Learning Loop

## Simple problem

In [10]:
learning_rate = 0.01

w = torch.tensor([1.], requires_grad = True)
b = torch.tensor([1.], requires_grad = True)

print(w.item(), b.item())

for i in range(10):

  x = torch.randn([20, 1])
  y = 3*x - 2

  y_hat = w*x + b
  loss = torch.sum((y_hat - y)**2)

  loss.backward()

  with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

    w.grad.zero_()
    b.grad.zero_()

  print(w.item(), b.item())

1.0 1.0
1.6551927328109741 -0.23028838634490967
1.9809269905090332 -0.960058331489563
2.5853943824768066 -1.6332018375396729
2.7290334701538086 -1.7757880687713623
2.8488235473632812 -1.8372719287872314
2.8656508922576904 -1.8780826330184937
2.9573276042938232 -1.9461849927902222
2.975231170654297 -1.9688034057617188
2.980072259902954 -1.9794526100158691
2.981700897216797 -1.9843369722366333


Note values of w and b have come close to the actual values (3 and -2)

## Large problem

In [0]:
%%time
learning_rate = 0.001
N = 1000
epochs = 2000

w = torch.rand([N], requires_grad=True)
b = torch.ones([1], requires_grad=True)

print(torch.mean(w).item(), b.item())

for i in range(epochs):

  x = torch.randn([N])
  y = torch.dot(3*torch.ones([N]),x) - 2

  y_hat = torch.dot(w,x) + b
  loss = torch.sum((y_hat - y)**2)

  loss.backward()

  with torch.no_grad():
    w -= learning_rate*w.grad
    b -= learning_rate*b.grad

    w.grad.zero_()
    b.grad.zero_()

  print(torch.mean(w).item(), b.item())

In [18]:
%%time
learning_rate = 0.001
N = 1000000
epochs = 200

w = torch.rand([N], requires_grad=True)
b = torch.ones([1], requires_grad=True)

# print(torch.mean(w).item(), b.item())

for i in range(epochs):

  x = torch.randn([N])
  y = torch.dot(3*torch.ones([N]),x) - 2

  y_hat = torch.dot(w,x) + b
  loss = torch.sum((y_hat - y)**2)

  loss.backward()

  with torch.no_grad():
    w -= learning_rate*w.grad
    b -= learning_rate*b.grad

    w.grad.zero_()
    b.grad.zero_()

  # print(torch.mean(w).item(), b.item())

CPU times: user 2.21 s, sys: 37.5 ms, total: 2.25 s
Wall time: 2.26 s


In [20]:
%%time
learning_rate = 0.001
N = 1000000
epochs = 200

w = torch.rand([N], requires_grad=True, device = cuda0)
b = torch.ones([1], requires_grad=True, device = cuda0)

# print(torch.mean(w).item(), b.item())

for i in range(epochs):

  x = torch.randn([N], device = cuda0)
  y = torch.dot(3*torch.ones([N], device = cuda0),x) - 2

  y_hat = torch.dot(w,x) + b
  loss = torch.sum((y_hat - y)**2)

  loss.backward()

  with torch.no_grad():
    w -= learning_rate*w.grad
    b -= learning_rate*b.grad

    w.grad.zero_()
    b.grad.zero_()

  # print(torch.mean(w).item(), b.item())

CPU times: user 145 ms, sys: 18.2 ms, total: 163 ms
Wall time: 165 ms
